# EDA – KoBART 기반 Dialogue Summarization

이 노트북은 **KoBART 백본 실험용** EDA 노트북입니다.
- 데이터 구조 및 길이/turn 분포
- KoBART 토크나이저 기준 토큰 길이 분포
- train/dev vs KoBART prediction 길이 분포 비교
- 요약 스타일(키워드) 확인


## 1. 데이터 로딩

- notebooks/ 폴더에서 실행해도 항상 프로젝트 루트의 `data/`를 바라보도록 경로를 처리합니다.

In [ ]:
# 데이터 로딩 셀입니다.
# - 이 노트북이 어디에서 실행되든 프로젝트 루트(nlp_contest)/data 를 기준으로 CSV를 읽습니다.
import pandas as pd
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
ROOT_DIR = NOTEBOOK_DIR.parent  # nlp_contest/
DATA_DIR = ROOT_DIR / "data"

TRAIN_PATH = DATA_DIR / "train.csv"
DEV_PATH = DATA_DIR / "dev.csv"
TEST_PATH = DATA_DIR / "test.csv"

missing = [p for p in [TRAIN_PATH, DEV_PATH, TEST_PATH] if not p.exists()]
if missing:
    missing_str = ", ".join(str(p) for p in missing)
    raise FileNotFoundError(
        f"다음 파일을 찾을 수 없습니다: {missing_str}\n"
        "- 프로젝트 루트(nlp_contest) 아래 data/에 csv를 넣은 뒤 이 셀을 다시 실행하세요."
    )

train_df = pd.read_csv(TRAIN_PATH)
dev_df = pd.read_csv(DEV_PATH)
test_df = pd.read_csv(TEST_PATH)

train_df.head(), dev_df.head(), test_df.head()

## 2. 기본 통계 및 길이/turn 분포

- 전체 크기, 컬럼, 길이/turn 분포를 KoBART 실험 관점에서 확인합니다.

In [ ]:
# 데이터 크기, 컬럼, topic 분포를 간단히 확인하는 셀입니다.
print("Train shape:", train_df.shape)
print("Dev shape:", dev_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain columns:", train_df.columns.tolist())
print("Dev columns:", dev_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

print("\nTrain topic value_counts:\n", train_df.get("topic", pd.Series()).value_counts().head())
print("\nDev topic value_counts:\n", dev_df.get("topic", pd.Series()).value_counts().head())

In [ ]:
# 길이(문자 기준)와 turn 수를 계산하는 셀입니다.
import re

for split_name, df in [("train", train_df), ("dev", dev_df)]:
    df["dialogue_len_char"] = df["dialogue"].astype(str).str.len()
    df["summary_len_char"] = df.get("summary", "").astype(str).str.len()

def count_turns(text: str) -> int:
    return len(re.findall(r"#Person[0-9]+#", str(text)))

train_df["num_turns"] = train_df["dialogue"].apply(count_turns)
dev_df["num_turns"] = dev_df["dialogue"].apply(count_turns)

print("[TRAIN] dialogue_len_char describe:\n", train_df["dialogue_len_char"].describe())
print("[DEV]   dialogue_len_char describe:\n", dev_df["dialogue_len_char"].describe())
print("\n[TRAIN] num_turns describe:\n", train_df["num_turns"].describe())
print("[DEV]   num_turns describe:\n", dev_df["num_turns"].describe())

## 3. KoBART 토크나이저 기준 토큰 길이 분포

- KoBART 토크나이저로 dialogue를 토크나이즈했을 때 토큰 길이 분포를 확인합니다.
- encoder_max_len=1024 설정이 얼마나 여유 있는지, 잘리는 비율이 어느 정도인지 보는 것이 목적입니다.

In [ ]:
# KoBART 토크나이저 기준 토큰 길이 분포를 그리는 셀입니다.
from transformers import PreTrainedTokenizerFast
import matplotlib.pyplot as plt
import seaborn as sns

kobart_tok = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v1")

def token_len(text: str) -> int:
    return len(kobart_tok.encode(str(text), add_special_tokens=True))

train_df["dialogue_len_tok"] = train_df["dialogue"].apply(token_len)
dev_df["dialogue_len_tok"] = dev_df["dialogue"].apply(token_len)

plt.figure(figsize=(8, 4))
sns.histplot(train_df["dialogue_len_tok"], bins=50, kde=True)
plt.axvline(1024, color="red", linestyle="--", label="encoder_max_len=1024")
plt.title("Train dialogue token length distribution (KoBART)")
plt.xlabel("dialogue_len_tok")
plt.ylabel("count")
plt.legend()
plt.show()

print("[TRAIN] encoder_max_len=1024 초과 비율:", (train_df["dialogue_len_tok"] > 1024).mean())
print("[DEV]   encoder_max_len=1024 초과 비율:", (dev_df["dialogue_len_tok"] > 1024).mean())

## 4. train vs dev 길이 분포 비교 (KDE)

- train/dev의 대화 길이(문자 기준) 분포를 KDE로 비교합니다.

In [ ]:
# train/dev dialogue_len_char 분포를 KDE로 비교하는 셀입니다.
plt.figure(figsize=(8, 4))
sns.kdeplot(train_df["dialogue_len_char"], label="train", shade=True)
sns.kdeplot(dev_df["dialogue_len_char"], label="dev", shade=True)
plt.title("dialogue_len_char distribution: train vs dev")
plt.xlabel("dialogue_len_char")
plt.legend()
plt.show()

## 5. KoBART prediction sanity check 및 길이 분포

- 현재 KoBART 실험에서 생성된 prediction CSV를 읽어 형식/길이 분포를 확인합니다.
- 파일 경로는 실제 실험 결과에 맞게 수정해서 사용하세요.

In [ ]:
# KoBART prediction CSV를 읽어서 기본 형식과 summary 길이 분포를 확인하는 셀입니다.
PRED_PATH = ROOT_DIR / "prediction" / "2511292249_kobart-base-style_prompt_bs8.csv"  # 필요시 최신 파일로 수정

if not PRED_PATH.exists():
    print(f"prediction 파일을 찾을 수 없습니다: {PRED_PATH}")
else:
    pred_df = pd.read_csv(PRED_PATH)
    print(pred_df.head())
    print("rows:", len(pred_df))
    print("fname unique:", pred_df["fname"].nunique())
    empty_ratio = (pred_df["summary"].astype(str).str.strip() == "").mean()
    print("empty summary 비율:", empty_ratio)
    print("summary 길이 통계:\n", pred_df["summary"].astype(str).str.len().describe())

    plt.figure(figsize=(8, 4))
    sns.kdeplot(train_df["summary_len_char"], label="gold train summary", shade=True)
    sns.kdeplot(pred_df["summary"].astype(str).str.len(), label="KoBART pred summary", shade=True)
    plt.title("summary length: gold vs KoBART prediction")
    plt.xlabel("summary length (chars)")
    plt.legend()
    plt.show()